In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import copy
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

if torch.cuda.is_available():
    print('CUDA is available. Working on GPU')
    DEVICE = torch.device('cuda')
else:
    print('CUDA is not available. Working on CPU')
    DEVICE = torch.device('cpu')

CUDA is available. Working on GPU


In [2]:
full_df = pd.read_csv("../labels_train_updated.csv")
df_region = full_df[['filename', 'Region_ID']]
print(df_region.head())

       filename  Region_ID
0  img_0000.jpg          2
1  img_0001.jpg          2
2  img_0002.jpg          2
3  img_0003.jpg          2
4  img_0004.jpg          2


In [3]:
max_region_row = df_region.loc[df_region['Region_ID'].idxmax()]
max_region = max_region_row['Region_ID']
filename = max_region_row['filename']

print("Max region id from the dataset:", max_region)
#print("Filename of the max angle image:", filename)


Max region id from the dataset: 15


In [4]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(10, translate=(0.05, 0.05)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [5]:
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [6]:
import torch
import torch.nn as nn
import timm  # make sure `timm` is installed

In [7]:
from sklearn.preprocessing import LabelEncoder

class RegionDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None, encoder=None, fit_encoder=False):
        self.df = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform
        
        if encoder is None:
            self.encoder = LabelEncoder()
        else:
            self.encoder = encoder
        
        if fit_encoder:
            self.encoder.fit(self.df['Region_ID'])
        
        self.df['Region_ID'] = self.encoder.transform(self.df['Region_ID'])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.df.iloc[idx, 0])
        region = self.df.iloc[idx, 5]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, region


In [8]:
class ResNetClassifier(nn.Module):
    def __init__(self, num_classes, backbone='resnet101'):
        super().__init__()
        self.backbone = timm.create_model(backbone, pretrained=True, num_classes=0)
        in_features = self.backbone.num_features

        self.classifier = nn.Sequential(
            nn.Linear(in_features, 2048),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        features = self.backbone(x)
        return self.classifier(features)


In [9]:
class DenseNetClassifier(nn.Module):
    def __init__(self, num_classes, backbone='densenet121'):
        super().__init__()
        self.backbone = timm.create_model(backbone, pretrained=True, num_classes=0, features_only=True)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        
        in_features = self.backbone.feature_info[-1]['num_chs']

        self.classifier = nn.Sequential(
            nn.Linear(in_features, 2048),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        features = self.backbone(x)[-1]
        pooled = self.pool(features)
        flattened = pooled.view(pooled.size(0), -1)
        return self.classifier(flattened)


In [10]:
class EfficientNetRegionClassifier(nn.Module):
    def __init__(self, num_classes, backbone='efficientnet_b3'):
        super().__init__()
        self.backbone = timm.create_model(backbone, pretrained=True, num_classes=0)
        in_features = self.backbone.num_features
        
        self.classifier = nn.Sequential(
        nn.Linear(in_features, 2048),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(2048, 1024),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, num_classes)
    )



    def forward(self, x):
        features = self.backbone(x)
        return self.classifier(features)


In [11]:
def evaluate_model(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    model.train()
    return 100 * correct / total


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import timm
from torchvision import transforms
from sklearn.preprocessing import LabelEncoder

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

encoder = LabelEncoder()

train_dataset = RegionDataset("../labels_train_updated.csv", '../images_train/images_train',
                              transform=transform, encoder=encoder, fit_encoder=True)

val_dataset = RegionDataset("../labels_val_updated.csv", "../images_val",
                            transform=val_transform, encoder=encoder, fit_encoder=False)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

num_classes = 15


In [13]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        print(f"✅ Epoch {epoch+1}, Loss: {avg_loss:.4f}")

        if (epoch + 1) % 5 == 0:
            val_acc = evaluate_model(model, val_loader, device)
            print(f"📊 Validation Accuracy after Epoch {epoch+1}: {val_acc:.2f}%")


In [21]:
best_acc = 0.0
best_model_state = None
best_lr = None
results = {}

for lr in [5e-4, 1e-4,3e-4, 1e-5]:
    print(f"\nTraining with learning rate: {lr}")

    resnet_model = ResNetClassifier(num_classes, backbone='resnet101')
    resnet_model.to(device)
    torch.cuda.empty_cache()
    optimizer = torch.optim.Adam(resnet_model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    train_model(resnet_model, train_loader, val_loader, criterion, optimizer, device, epochs=15)

    final_acc = evaluate_model(resnet_model, val_loader, device)
    results[lr] = final_acc
    print(f"✅ Validation Accuracy for LR={lr}: {final_acc:.2f}%")
    if final_acc > best_acc:
        best_acc = final_acc
        best_lr = lr
        best_model_state = copy.deepcopy(resnet_model.state_dict())  # deep copy model weights

torch.save(best_model_state, f'best_resnet_model_lr_{best_lr}.pt')
print(f"\n🏆 Best LR: {best_lr} with Accuracy: {best_acc:.2f}% — Model saved!")



Training with learning rate: 0.0005


Epoch 1/15: 100%|██████████| 818/818 [03:24<00:00,  4.01it/s]


✅ Epoch 1, Loss: 2.4292


Epoch 2/15: 100%|██████████| 818/818 [03:12<00:00,  4.25it/s]


✅ Epoch 2, Loss: 1.8417


...

✅ Epoch 13, Loss: 2.1278


Epoch 14/15: 100%|██████████| 818/818 [03:03<00:00,  4.45it/s]


✅ Epoch 14, Loss: 2.0893


Epoch 15/15: 100%|██████████| 818/818 [03:01<00:00,  4.52it/s]


✅ Epoch 15, Loss: 2.0343
📊 Validation Accuracy after Epoch 15: 31.44%
✅ Validation Accuracy for LR=1e-05: 31.44%

🏆 Best LR: 0.0005 with Accuracy: 80.76% — Model saved!


In [28]:
best_acc = 0.0
best_model_state = None
best_lr = None
results = {}

for lr in [3e-4,2e-4,1e-4,3e-3,1e-3]:
    print(f"\nTraining with learning rate: {lr}")

    efficientnet_model = EfficientNetRegionClassifier(num_classes, backbone='efficientnet_b0')
    efficientnet_model.to(device)
    torch.cuda.empty_cache()
    optimizer = torch.optim.Adam(efficientnet_model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    train_model(efficientnet_model, train_loader, val_loader, criterion, optimizer, device, epochs=15)

    final_acc = evaluate_model(efficientnet_model, val_loader, device)
    results[lr] = final_acc
    print(f"✅ Validation Accuracy for LR={lr}: {final_acc:.2f}%")
    if final_acc > best_acc:
        best_acc = final_acc
        best_lr = lr
        best_model_state = copy.deepcopy(efficientnet_model.state_dict())  # deep copy model weights

torch.save(best_model_state, f'best_efficientnet_model_lr_{best_lr}.pt')
print(f"\n🏆 Best LR: {best_lr} with Accuracy: {best_acc:.2f}% — Model saved!")



Training with learning rate: 0.0003


pytorch_model.bin:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

Epoch 1/15: 100%|██████████| 818/818 [01:44<00:00,  7.84it/s]


✅ Epoch 1, Loss: 2.0766


Epoch 2/15: 100%|██████████| 818/818 [01:52<00:00,  7.30it/s]


...

✅ Epoch 13, Loss: 1.5097


Epoch 14/15: 100%|██████████| 818/818 [01:46<00:00,  7.69it/s]


✅ Epoch 14, Loss: 1.4663


Epoch 15/15: 100%|██████████| 818/818 [01:50<00:00,  7.42it/s]


✅ Epoch 15, Loss: 1.4727
📊 Validation Accuracy after Epoch 15: 43.36%
✅ Validation Accuracy for LR=0.001: 43.36%

🏆 Best LR: 0.0001 with Accuracy: 83.47% — Model saved!


In [14]:
best_acc = 0.0
best_model_state = None
best_lr = None
results = {}

for lr in [1e-4,2e-4,3e-4, 1e-5,]:
    print(f"\nTraining with learning rate: {lr}")

    eff2_model = EfficientNetRegionClassifier(num_classes, backbone='efficientnet_b1')
    eff2_model.to(device)
    torch.cuda.empty_cache()
    optimizer = torch.optim.Adam(eff2_model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    train_model(eff2_model, train_loader, val_loader, criterion, optimizer, device, epochs=15)

    final_acc = evaluate_model(eff2_model, val_loader, device)
    results[lr] = final_acc
    print(f"✅ Validation Accuracy for LR={lr}: {final_acc:.2f}%")
    if final_acc > best_acc:
        best_acc = final_acc
        best_lr = lr
        best_model_state = copy.deepcopy(eff2_model.state_dict())  # deep copy model weights

torch.save(best_model_state, f'best_eff2_model_lr_{best_lr}.pt')
print(f"\n🏆 Best LR: {best_lr} with Accuracy: {best_acc:.2f}% — Model saved!")


Training with learning rate: 0.0001


Epoch 1/15: 100%|██████████| 818/818 [01:44<00:00,  7.84it/s]


✅ Epoch 1, Loss: 2.3261


Epoch 2/15: 100%|██████████| 818/818 [01:40<00:00,  8.13it/s]


✅ Epoch 2, Loss: 1.8263


...

✅ Epoch 13, Loss: 0.5905


Epoch 14/15: 100%|██████████| 818/818 [01:47<00:00,  7.61it/s]


✅ Epoch 14, Loss: 0.5402


Epoch 15/15: 100%|██████████| 818/818 [01:47<00:00,  7.63it/s]


✅ Epoch 15, Loss: 0.4900
📊 Validation Accuracy after Epoch 15: 74.80%
✅ Validation Accuracy for LR=1e-05: 74.80%

🏆 Best LR: 1e-05 with Accuracy: 74.80% — Model saved!


In [17]:
best_acc = 0.0
best_model_state = None
best_lr = None
results = {}

for lr in [5e-4, 2e-4,1e-4,3e-4, 1e-5]:
    print(f"\nTraining with learning rate: {lr}")

    res2_model = ResNetClassifier(num_classes, backbone='resnet50')
    res2_model.to(device)
    torch.cuda.empty_cache()
    optimizer = torch.optim.Adam(res2_model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    train_model(res2_model, train_loader, val_loader, criterion, optimizer, device, epochs=15)

    final_acc = evaluate_model(res2_model, val_loader, device)
    results[lr] = final_acc
    print(f"✅ Validation Accuracy for LR={lr}: {final_acc:.2f}%")
    if final_acc > best_acc:
        best_acc = final_acc
        best_lr = lr
        best_model_state = copy.deepcopy(res2_model.state_dict())  # deep copy model weights

torch.save(best_model_state, f'best_res2_model_lr_{best_lr}.pt')
print(f"\n🏆 Best LR: {best_lr} with Accuracy: {best_acc:.2f}% — Model saved!")


Training with learning rate: 0.0005


Epoch 1/15: 100%|██████████| 818/818 [02:02<00:00,  6.67it/s]


✅ Epoch 1, Loss: 2.3770


Epoch 2/15: 100%|██████████| 818/818 [02:02<00:00,  6.67it/s]


✅ Epoch 2, Loss: 1.7874


...

✅ Epoch 13, Loss: 1.8512


Epoch 14/15: 100%|██████████| 818/818 [02:03<00:00,  6.63it/s]


✅ Epoch 14, Loss: 1.8310


Epoch 15/15: 100%|██████████| 818/818 [02:03<00:00,  6.65it/s]


✅ Epoch 15, Loss: 1.7916
📊 Validation Accuracy after Epoch 15: 34.42%
✅ Validation Accuracy for LR=1e-05: 34.42%

🏆 Best LR: 0.0001 with Accuracy: 80.49% — Model saved!


In [18]:
# Load models
effnet_b0 = EfficientNetRegionClassifier(num_classes, backbone='efficientnet_b0')
effnet_b0.load_state_dict(torch.load('best_efficientnet_model_lr_0.0001.pt'))
effnet_b0.to(device).eval()


effnet_b1 = EfficientNetRegionClassifier(num_classes, backbone='efficientnet_b1')
effnet_b1.load_state_dict(torch.load('best_eff2_model_lr_1e-05.pt'))
effnet_b1.eval().to(device)

resnet50 = ResNetClassifier(num_classes, backbone='resnet50')
resnet50.load_state_dict(torch.load('best_res2_model_lr_0.0001.pt'))
resnet50.eval().to(device)

resnet101 = ResNetClassifier(num_classes, backbone='resnet101')
resnet101.load_state_dict(torch.load('best_resnet_model_lr_0.0005.pt'))
resnet101.eval().to(device)


/tmp/ipykernel_6970/2648064454.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  effnet_b0.load_state_dict(torch.load('best_efficientnet_model_lr_0.0001.pt'))
/tmp/ipykern

ResNetClassifier(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act2): ReLU(inplace=True)
        (aa): Identity()
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1

In [20]:
class EnsembleModel(nn.Module):
    def __init__(self, models):
        super().__init__()
        self.models = models

    def forward(self, x):
        outputs = [F.softmax(model(x), dim=1) for model in self.models]
        return torch.stack(outputs).mean(0)  # average softmax predictions


In [21]:
def evaluate(model, dataloader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return 100.0 * correct / total


In [42]:
models = [effnet_b0, effnet_b1, resnet50, resnet101]
ensemble_model = EnsembleModel(models).to(device)
val_accuracy = evaluate(ensemble_model, val_loader)
print(f"✅ Ensemble Validation Accuracy: {val_accuracy:.2f}%")

✅ Ensemble Validation Accuracy: 88.08%


In [46]:
class WeightedEnsembleLogits(nn.Module):
    def __init__(self, models, weights):
        super().__init__()
        self.models = models
        self.weights = weights

    def forward(self, x):
        # Get logits from each model
        logits = [model(x) for model in self.models]
        # Weight and sum the logits
        weighted_logits = [w * logit for w, logit in zip(self.weights, logits)]
        return torch.stack(weighted_logits).sum(0)  # weighted sum of logits

# Create ensemble model
weights = [0.5, 0.4, 0.3, 0.2]  # weights based on performance (EffNet-B0 > ResNet50 > ResNet101 > EffNet-B1)
ensemble_model = WeightedEnsembleLogits(models=[effnet_b0, resnet50, resnet101, effnet_b1], weights=weights)

# Evaluate the model
accuracy = evaluate(ensemble_model, val_loader)
print(f"Ensemble Model Accuracy: {accuracy:.2f}%")

Ensemble Model Accuracy: 89.70%


In [27]:
import itertools
import numpy as np

def find_best_weights(models, val_loader):
    best_acc = 0
    best_weights = None
    all_weights = [w for w in itertools.product(np.arange(0.1, 1.1, 0.1), repeat=len(models)) if np.isclose(sum(w), 1.0)]
    
    for weights in all_weights:
        ensemble = WeightedEnsembleLogits(models=models, weights=weights)
        acc = evaluate(ensemble, val_loader)
        if acc > best_acc:
            best_acc = acc
            best_weights = weights
            print(f"New best accuracy: {acc:.2f}% with weights: {weights}")
    
    return best_weights, best_acc


In [ ]:
best_weights, best_accuracy = find_best_weights([effnet_b0, resnet50, resnet101, effnet_b1], val_loader)
print("Best Weights:", best_weights)


New best accuracy: 84.01% with weights: (0.1, 0.1, 0.1, 0.7000000000000001)
New best accuracy: 86.99% with weights: (0.1, 0.1, 0.2, 0.6)
New best accuracy: 87.53% with weights: (0.1, 0.1, 0.30000000000000004, 0.5)
New best accuracy: 88.35% with weights: (0.1, 0.2, 0.2, 0.5)
New best accuracy: 89.43% with weights: (0.1, 0.2, 0.30000000000000004, 0.4)
New best accuracy: 89.70% with weights: (0.2, 0.2, 0.2, 0.4)
New best accuracy: 90.24% with weights: (0.30000000000000004, 0.1, 0.2, 0.4)
New best accuracy: 90.51% with weights: (0.4, 0.30000000000000004, 0.1, 0.2)
Best Weights: (0.4, 0.30000000000000004, 0.1, 0.2)


In [49]:
weights = [0.4, 0.30000000000000004, 0.1, 0.2]  # weights based on performance (EffNet-B0 > ResNet50 > ResNet101 > EffNet-B1)
ensemble_model = WeightedEnsembleLogits(models=[effnet_b0, resnet50, resnet101, effnet_b1], weights=weights)

# Evaluate the model
accuracy = evaluate(ensemble_model, val_loader)
print(f"Ensemble Model Accuracy: {accuracy:.2f}%")

Ensemble Model Accuracy: 90.51%


In [14]:
best_acc = 0.0
best_model_state = None
best_lr = None
results = {}

for lr in [3e-4,2e-4,1e-4,3e-3,1e-3]:
    print(f"\nTraining with learning rate: {lr}")

    densenet_model = DenseNetClassifier(num_classes, backbone='densenet121')
    densenet_model.to(device)
    torch.cuda.empty_cache()
    optimizer = torch.optim.Adam(densenet_model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    train_model(densenet_model, train_loader, val_loader, criterion, optimizer, device, epochs=15)

    final_acc = evaluate_model(densenet_model, val_loader, device)
    results[lr] = final_acc
    print(f"✅ Validation Accuracy for LR={lr}: {final_acc:.2f}%")
    if final_acc > best_acc:
        best_acc = final_acc
        best_lr = lr
        best_model_state = copy.deepcopy(densenet_model.state_dict())  # deep copy model weights

torch.save(best_model_state, f'best_densenet_model_lr_{best_lr}.pt')
print(f"\n🏆 Best LR: {best_lr} with Accuracy: {best_acc:.2f}% — Model saved!")



Training with learning rate: 0.0003


model.safetensors:  32%|###2      | 10.5M/32.3M [00:00<?, ?B/s]

Epoch 1/15: 100%|██████████| 818/818 [02:42<00:00,  5.04it/s]


✅ Epoch 1, Loss: 2.3019


Epoch 2/15: 100%|██████████| 818/818 [02:33<00:00,  5.33it/s]


...

✅ Epoch 13, Loss: 2.0480


Epoch 14/15: 100%|██████████| 818/818 [02:26<00:00,  5.58it/s]


✅ Epoch 14, Loss: 1.9950


Epoch 15/15: 100%|██████████| 818/818 [02:26<00:00,  5.58it/s]


✅ Epoch 15, Loss: 1.9235
📊 Validation Accuracy after Epoch 15: 32.79%
✅ Validation Accuracy for LR=0.001: 32.79%

🏆 Best LR: 0.0001 with Accuracy: 83.20% — Model saved!


In [16]:
densenet_121 = DenseNetClassifier(num_classes, backbone='densenet121')
densenet_121.load_state_dict(torch.load('best_densenet_model_lr_0.0001.pt'))
densenet_121.to(device).eval()

/tmp/ipykernel_6970/4089935998.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  densenet_121.load_state_dict(torch.load('best_densenet_model_lr_0.0001.pt'))


DenseNetClassifier(
  (backbone): FeatureListNet(
    (features_conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (features_norm0): BatchNormAct2d(
      64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
    (features_pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (features_denseblock1): DenseBlock(
      (denselayer1): DenseLayer(
        (norm1): BatchNormAct2d(
          64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNormAct2d(
          128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), str

In [22]:
models = [effnet_b0, densenet_121, resnet50, resnet101]
ensemble_model = EnsembleModel(models).to(device)
val_accuracy = evaluate(ensemble_model, val_loader)
print(f"✅ Ensemble Validation Accuracy: {val_accuracy:.2f}%")

✅ Ensemble Validation Accuracy: 88.35%
